In [8]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
folder_path = r'C:\Users\Nadeem\Desktop\BSDS\Semester 6\Artificial Intelligence\flowers'

data = []
labels = []

image_size = (64, 64)

for folder_name in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, folder_name)
    for file_name in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, file_name)
        img = Image.open(file_path)
        img = img.resize(image_size)
        img_array = np.array(img)
        data.append(img_array)
        labels.append(folder_name)

X = np.array(data)
y = np.array(labels)

In [12]:
print(X)
print(y)

[[[[141 141 139]
   [149 148 149]
   [152 152 154]
   ...
   [163 163 167]
   [155 155 154]
   [152 152 151]]

  [[136 136 133]
   [146 145 145]
   [163 162 167]
   ...
   [159 158 162]
   [156 156 154]
   [149 148 148]]

  [[129 128 122]
   [139 140 136]
   [154 154 155]
   ...
   [157 157 157]
   [155 155 154]
   [141 140 139]]

  ...

  [[ 40  43  21]
   [ 41  44  22]
   [ 55  57  42]
   ...
   [128 124 122]
   [126 122 119]
   [126 123 118]]

  [[ 43  46  24]
   [ 44  47  25]
   [ 52  54  36]
   ...
   [133 129 128]
   [129 125 124]
   [128 125 123]]

  [[ 44  47  26]
   [ 46  48  27]
   [ 52  54  35]
   ...
   [137 133 132]
   [133 129 128]
   [130 126 125]]]


 [[[216 218 226]
   [216 219 226]
   [219 221 228]
   ...
   [ 18  24  22]
   [ 14  16  17]
   [  9   9  14]]

  [[229 231 239]
   [224 227 233]
   [220 222 229]
   ...
   [ 24  29  28]
   [ 15  17  18]
   [  9   8  13]]

  [[229 233 239]
   [232 236 243]
   [228 233 240]
   ...
   [ 32  36  34]
   [ 15  17  19]
   [  9   8

In [17]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [15]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros(output_size)

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = np.exp(self.Z2) / np.sum(np.exp(self.Z2), axis=1, keepdims=True)

    def backward(self, X, y):
        m = len(y)

        dZ2 = self.A2
        dZ2[np.arange(m), y] -= 1
        dW2 = np.dot(self.A1.T, dZ2)
        db2 = np.sum(dZ2, axis=0)
        dZ1 = np.dot(dZ2, self.W2.T) * self.sigmoid_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1)
        db1 = np.sum(dZ1, axis=0)

        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1

    def train(self, X, y, learning_rate=0.1, epochs=1000, verbose=True):
        self.learning_rate = learning_rate
        self.epochs = epochs

        y = y.astype(int)

        for epoch in range(1, self.epochs + 1):
            self.forward(X)
            loss = self.loss(y)

            if verbose and epoch % 100 == 0:
                print('Epoch:', epoch, 'Loss:', loss)

            self.backward(X, y)

    def predict(self, X):
        self.forward(X)
        return np.argmax(self.A2, axis=1)

    def loss(self, y):
        y = y.astype(int)
        return -np.sum(np.log(self.A2[np.arange(len(y)), y])) / len(y)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

X_train_norm = X_train_flat / 255
X_test_norm = X_test_flat / 255

input_size = X_train_norm.shape[1]
hidden_size = 100
output_size = len(np.unique(y_train))
learning_rate = 0.1
epochs = 10

mlp = MLP(input_size, hidden_size, output_size)

mlp.train(X_train_norm, y_train, learning_rate=learning_rate, epochs=epochs)

y_pred = mlp.predict(X_test_norm)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

<ipython-input-15-f9350de033dd>:58: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
<ipython-input-15-f9350de033dd>:16: RuntimeWarning: overflow encountered in exp
  self.A2 = np.exp(self.Z2) / np.sum(np.exp(self.Z2), axis=1, keepdims=True)
<ipython-input-15-f9350de033dd>:16: RuntimeWarning: invalid value encountered in true_divide
  self.A2 = np.exp(self.Z2) / np.sum(np.exp(self.Z2), axis=1, keepdims=True)
<ipython-input-15-f9350de033dd>:55: RuntimeWarning: divide by zero encountered in log
  return -np.sum(np.log(self.A2[np.arange(len(y)), y])) / len(y)


Epoch: 100 Loss: nan
Accuracy: 0.1875
